In [2]:
# Import Required Libraries
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt

In [3]:
# Load the horses_or_humans dataset
(train_data, test_data), info = tfds.load(
    'horses_or_humans',
    with_info=True,
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1027 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/horses_or_humans/3.0.0.incompleteGHD8H2/horses_or_humans-train.tfrecord*..…

Generating test examples...:   0%|          | 0/256 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/horses_or_humans/3.0.0.incompleteGHD8H2/horses_or_humans-test.tfrecord*...…

Dataset horses_or_humans downloaded and prepared to /root/tensorflow_datasets/horses_or_humans/3.0.0. Subsequent calls will reuse this data.


In [4]:
# Explore the dataset
for images, labels in train_data.take(1):
    print("Images shape: ", images.shape)
    print("Labels shape: ", labels.shape)

Images shape:  (300, 300, 3)
Labels shape:  ()


In [5]:
# Create a convolutional neural network (CNN)
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 186624)            0         
                                                                 
 dense (Dense)               (None, 128)               2

In [6]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
train_ds = (train_data
             .map(lambda img, label: (tf.image.resize(img, [224, 224]), label))
             .batch(32)
             .prefetch(1))

test_ds = (test_data
           .map(lambda img, label: (tf.image.resize(img, [224, 224]), label))
           .batch(32)
           .prefetch(1))

In [8]:
# Train the model
history = model.fit(train_ds, epochs=10, validation_data=test_ds)

Epoch 1/10
33/33 [==============================] - 12s 174ms/step - loss: 295.9165 - accuracy: 0.7770 - val_loss: 0.7108 - val_accuracy: 0.8789
Epoch 2/10
33/33 [==============================] - 5s 154ms/step - loss: 0.0393 - accuracy: 0.9893 - val_loss: 1.1187 - val_accuracy: 0.8906
Epoch 3/10
33/33 [==============================] - 4s 125ms/step - loss: 0.0053 - accuracy: 0.9981 - val_loss: 1.9087 - val_accuracy: 0.8398
Epoch 4/10
33/33 [==============================] - 4s 122ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 3.0881 - val_accuracy: 0.8359
Epoch 5/10
33/33 [==============================] - 5s 140ms/step - loss: 4.3561e-05 - accuracy: 1.0000 - val_loss: 2.9241 - val_accuracy: 0.8398
Epoch 6/10
33/33 [==============================] - 4s 122ms/step - loss: 2.2027e-06 - accuracy: 1.0000 - val_loss: 3.0100 - val_accuracy: 0.8398
Epoch 7/10
33/33 [==============================] - 6s 177ms/step - loss: 1.2047e-06 - accuracy: 1.0000 - val_loss: 3.0384 - val_accuracy

In [9]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_ds)
print('Test accuracy:', test_acc)

8/8 [==============================] - 0s 18ms/step - loss: 3.0540 - accuracy: 0.8438
Test accuracy: 0.84375
